In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
!pip install flask_pymongo
from flask_pymongo import PyMongo
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Driver [C:\Users\clseg\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


# Step 1 - Scraping
# NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
Assign the text to variables that you can reference later.

In [3]:
# URL of page to be scraped -- Latest news
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

# Scrape page with Beautiful Soup
html=browser.html
soup=bs(html,'html.parser')

In [4]:
# Extract article title and paragraph
new_title = soup.find('div', class_="content_title").get_text(strip=True)
news_p = soup.find("div", class_="article_teaser_body").get_text(strip=True)
# article

# JPL Mars Space Images - Featured Image

In [5]:
# URl for images
image_url="https://spaceimages-mars.com/"
browser.visit(image_url)
time.sleep(1)

In [6]:
# Create soup object from HTML
html=browser.html
# Parse HTML
soup=bs(html,"html.parser")
# Retrieve image url
relative_image_path=soup.find('img', class_="headerimage fade-in")["src"]
featured_image_url= image_url + relative_image_path
featured_image_url


'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Mars Facts

In [7]:
# ULR for Mars Facts webpage 
facts_url = "https://galaxyfacts-mars.com/"
browser.visit(facts_url)
time.sleep(1)

In [8]:
# Create soup object from HTML
html=browser.html


In [9]:
# Pandas to scrape the table containing facts about Mars
table = pd.read_html(facts_url)
mars_facts = table[1]

# Rename columns
mars_facts.columns = ['Description','Value']

# Reset Index to be description
mars_facts.set_index('Description', inplace=True)
mars_facts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [12]:
# Use Pandas to convert to a HTML table
fact_table=mars_facts.to_html('table.html')


# Mars Hemispheres

In [24]:
# Visit URL to get Mars hemisphere images
hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)

In [25]:
#Container to hold  our loop iterations
all_urls = []

#Iterate through the 4 links to get the images needed
for x in range(4):
    html = browser.html
    soup = bs(html, "html.parser")
    
    title = soup.find_all("h3")[x].get_text()
    browser.find_by_tag("h3")[x].click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    img_url = soup.find("img", class_="wide-image")["src"]
    all_urls.append({
        "title":title,
        "img_url": hemi_url+img_url
    })
    browser.back()
all_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]